# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

Подключаем модули:

In [138]:
import itertools
import math
import numpy as np
import random

Читаем граф из файла:

In [151]:
def read_col_file(filename):
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

Функция подсчета числа ребер, пересекающих разрез. Проверяем, в одной ли компоненте лежат концы ребра.

In [172]:
def count_cut(colors, edges):
    return np.array(list(map(lambda edge: colors[edge[0]] != colors[edge[1]], edges))).sum()

Генерируем случайно две компоненты. Затем в каждой итерации меняем две случайные вершины. Если разрез увеличился, то меняем вершины местами обратно.

In [186]:
def basic_local_search(graph):
    size, vertices, edges = len(graph[0]), graph[0], list(graph[1])
    
    fragment = list(np.random.choice(list(vertices), size=len(vertices) // 2, replace=False))
    the_other_fragment = list(vertices - set(fragment))
    
    colors = [-1] * (size + 1)
    
    for v in fragment:
        colors[v] = 0
    for v in the_other_fragment:
        colors[v] = 1
        
    iteration = 0
    
    while iteration < 10000:
        random.shuffle(fragment)
        random.shuffle(the_other_fragment)
        now_cut = count_cut(colors, edges)
        fragment[0], the_other_fragment[0] = the_other_fragment[0], fragment[0]
        colors[fragment[0]], colors[the_other_fragment[0]] = colors[the_other_fragment[0]], colors[fragment[0]]
        if count_cut(colors, edges) > now_cut:
            fragment[0], the_other_fragment[0] = the_other_fragment[0], fragment[0]
            colors[fragment[0]], colors[the_other_fragment[0]] = colors[the_other_fragment[0]], colors[fragment[0]]
        iteration += 1
    
    return fragment, the_other_fragment, count_cut(colors, edges)

Привожу результат работы алгоритма на всех файлах:

In [187]:
for X in range (3, 8):
    filename = 'myciel' + str(X) + '.col.txt'
    V1, V2, cost = basic_local_search(read_col_file(filename))
    print('Тест под номером ' + str(X) + ':')
    print('Первая доля:', sorted(V1))
    print('Вторая доля:', sorted(V2))
    print('Величиная разреза:', cost, '\n')

Тест под номером 3:
Первая доля: [1, 6, 7, 9, 11]
Вторая доля: [2, 3, 4, 5, 8, 10]
Величиная разреза: 8 

Тест под номером 4:
Первая доля: [4, 5, 9, 11, 12, 16, 17, 19, 20, 21, 23]
Вторая доля: [1, 2, 3, 6, 7, 8, 10, 13, 14, 15, 18, 22]
Величиная разреза: 27 

Тест под номером 5:
Первая доля: [2, 3, 6, 7, 8, 10, 12, 13, 14, 16, 17, 18, 19, 21, 22, 25, 26, 27, 30, 31, 34, 45, 46]
Вторая доля: [1, 4, 5, 9, 11, 15, 20, 23, 24, 28, 29, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47]
Величиная разреза: 91 

Тест под номером 6:
Первая доля: [3, 4, 8, 9, 13, 15, 16, 17, 18, 19, 20, 21, 23, 24, 26, 28, 29, 31, 33, 35, 36, 39, 40, 41, 42, 43, 44, 46, 48, 49, 50, 51, 52, 58, 60, 62, 63, 65, 68, 69, 70, 71, 75, 81, 86, 93, 94]
Вторая доля: [1, 2, 5, 6, 7, 10, 11, 12, 14, 22, 25, 27, 30, 32, 34, 37, 38, 45, 47, 53, 54, 55, 56, 57, 59, 61, 64, 66, 67, 72, 73, 74, 76, 77, 78, 79, 80, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 95]
Величиная разреза: 289 

Тест под номером 7:
Первая доля: [2, 3, 4, 1